In [43]:
import numpy as np
import os, glob
import pandas as pd
from scipy.interpolate import interp1d

In [23]:
!python -m pip install scipy


In [87]:
data_path = os.path.join("Dataset", "Incorrect", "m01*.txt")
skels = []
sum = 0
for f in glob.glob(data_path):
    skel = pd.read_csv(f, delimiter=",", header=None)
    skel_np = skel.to_numpy()
    skel_3d = skel_np.reshape(22,-1,2).transpose(1, 0, 2)
    RUA = skel_3d[:,11,:]
    LUA = skel_3d[:,7,:]
    scale = np.sqrt(((RUA[:,0] - LUA[:,0]) ** 2) + ((RUA[:,1] - LUA[:,1]) ** 2))
    skels.append(np.divide(skel_3d, scale[:, None, None]))
    sum += skel_3d.shape[0]

length_mean = sum // len(skels)


In [88]:
skels_meanl = np.zeros([len(skels), 22, length_mean, 2])
for i, skel in enumerate(skels):
    xs = skel[:,:,0]
    ys = skel[:,:,1]
    xs_meanl = []
    ys_meanl = []
    for j in range(22):
        x = xs[:, j]
        y = ys[:, j]
        x_interp = interp1d(np.arange(x.size), x)
        y_interp = interp1d(np.arange(y.size), y)
        x_meanl = x_interp(np.linspace(0, x.size-1, length_mean))
        y_meanl = y_interp(np.linspace(0, y.size-1, length_mean))

        xs_meanl.append(x_meanl)
        ys_meanl.append(y_meanl)
        
##        plot interpolations
#         xmin, xmax = 0, 1
#         fig,(ax1,ax2) = plt.subplots(ncols=2)
#         ax1.plot(np.linspace(xmin,xmax,x.size),x,'bo-',np.linspace(xmin,xmax,x_meanl.size),x_meanl,'rs')
#         ax2.plot(np.linspace(xmin,xmax,y.size),y,'bo-',np.linspace(xmin,xmax,y_meanl.size),y_meanl,'rs') 
    

    skels_meanl[i, :, :, 0] = np.asarray(xs_meanl)
    skels_meanl[i, :, :, 1] = np.asarray(ys_meanl)



In [89]:
skels_meanl = skels_meanl.transpose(0, 1, 3, 2).reshape(len(skels)*22*2, length_mean)
np.savetxt(os.path.join("Dataset", "Incorrect_data.txt"), skels_meanl, delimiter=',')